<a href="https://colab.research.google.com/github/aabchyein/study_data_analystics/blob/main/codes/NLP/healthapp_review_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer : https://heytech.tistory.com/401

In [ ]:
# !pip install python-mecab-ko pyLDAvis

In [ ]:
import numpy as np

In [7]:
import pandas as pd
df_review_negative = pd.read_csv('./review_negative.csv')
df_review_negative[:2]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review
0,1072,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무 힘들다,1,NaN,NaN
1,1167,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,훌륭,1,NaN,NaN


In [8]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        413 non-null    int64 
 1   app               413 non-null    object
 2   review            413 non-null    object
 3   rating            413 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 19.5+ KB


In [9]:
# null 값 제거
df_review_negative = df_review_negative.dropna()
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 346 entries, 3 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        346 non-null    int64 
 1   app               346 non-null    object
 2   review            346 non-null    object
 3   rating            346 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


## LDA 분석
- negative review 분석

In [10]:
from gensim.corpora import Dictionary

In [11]:
df_review_negative[['replaced_review']]  # 2차원 배열

,replaced_review
3,칼로리
5,마음
7,오류
8,모야
9,느림
...,...
406,운동 리뷰 출창
407,광고 짜증
408,운동 친절 운동 운동 체계 순서 운동 기분 시작 운동 구간 시작
409,광고 운동


In [12]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values) # 2차원 배열에서 값만 추출

In [13]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3]) # 리뷰 하나의 값을 넣어서 벡터화 확인

[(3, 1)]

In [14]:
# dictionary에 의한 한글 문장을 벡터화
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [15]:
# 컬럼 추가해서 list를 담아줌
df_review_negative['문장 벡터화'] = corpus_list

In [16]:
df_review_negative.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장 벡터화'],
      dtype='object')

#### 토픽 잡기
- 최적의 토픽개수가 3개라고 가정

In [17]:
from gensim.models import LdaModel

In [18]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [19]:
lda_model.print_topics(num_words=4)
# 토픽에서 모든 단어를 다 볼 필요가 없기 때문에 4개의 단어까지만 나오게 지정

[(0, '0.013*"결제 환불" + 0.013*"결제 취소" + 0.010*"결제" + 0.009*"자동 결제 취소"'),
 (1, '0.024*"운동" + 0.012*"불편" + 0.009*"위젯" + 0.008*"자동 결제 환불"'),
 (2, '0.029*"로그인" + 0.025*"환불" + 0.009*"광고" + 0.009*"의견"')]

### LDA 시각화

In [ ]:
%pip install pyLDAvis

In [20]:
import pyLDAvis.gensim_models

In [21]:
pyLDAvis.enable_notebook()   # 일반 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [23]:
pyLDAvis.display(result_visualized)

# 동그라미 : 토픽의 개수 (간격이 적절하게 떨어져 있는 것이 좋음)

In [24]:
# 시각화를 html로 저장
pyLDAvis.save_html(result_visualized, './result_visualized.html')
# 깃허브에 올리면 html이라 바로 열어볼 수 있음 혹은 캡처를 해서 이미지로 저장해 리드미에 올려놓는 것이 좋음

# 퀘스트
- review_positive.csv 토픽 분석
- 토픽 수는 3으로 가정

## LDA 분석
- positive review 분석

In [27]:
df_review_positive = pd.read_csv('./review_positive.csv')
df_review_positive[:2]

,Unnamed: 0,app,review,rating,tokenized_review,replaced_review
0,1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,NaN,NaN
1,1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고


In [28]:
df_review_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7587 entries, 0 to 7586
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        7587 non-null   int64 
 1   app               7587 non-null   object
 2   review            7587 non-null   object
 3   rating            7587 non-null   int64 
 4   tokenized_review  4351 non-null   object
 5   replaced_review   4351 non-null   object
dtypes: int64(2), object(4)
memory usage: 355.8+ KB


In [32]:
# null 값 제거
df_review_positive = df_review_positive.dropna()
df_review_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4351 entries, 1 to 7585
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        4351 non-null   int64 
 1   app               4351 non-null   object
 2   review            4351 non-null   object
 3   rating            4351 non-null   int64 
 4   tokenized_review  4351 non-null   object
 5   replaced_review   4351 non-null   object
dtypes: int64(2), object(4)
memory usage: 237.9+ KB


In [33]:
from gensim.corpora import Dictionary

In [34]:
df_review_positive[['replaced_review']]

,replaced_review
1,최고
4,필요 운동 최고
5,운동 최고
7,집중 운동
12,운동 도움
...,...
7579,리뷰 설명 타임 운동 추천 건강
7580,운동 몸무게 점차 저녁 기대
7582,추천
7583,운동 영상


In [42]:
# 2차원 배열에서 값만 추출
dictionary = Dictionary(df_review_positive[['replaced_review']].values)

In [44]:
# 벡터화 확인
dictionary.doc2bow(df_review_positive[['replaced_review']].values[3])

[(3, 1)]

In [50]:
# dictionary에 의한 한글 문장을 벡터화
corpus_list = list()
for sentence in df_review_positive[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [51]:
# 컬럼 추가
df_review_positive['문장 벡터화'] = corpus_list

In [53]:
# 컬럼 추가 확인
df_review_positive.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장 벡터화'],
      dtype='object')

#### 토픽 잡기

In [54]:
from gensim.models import LdaModel

In [55]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [56]:
lda_model.print_topics(num_words=4)

[(0, '0.268*"운동" + 0.027*"도움" + 0.018*"운동 최고" + 0.015*"운동 도움"'),
 (1, '0.040*"뱃살" + 0.031*"추천" + 0.030*"감사" + 0.015*"강추"'),
 (2, '0.081*"최고" + 0.017*"광고" + 0.011*"유용" + 0.009*"트레이닝"')]

### LDA 시각화

In [ ]:
# %pip install pyLDAvis